In [4]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from torch import tensor
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import random
import os
import matplotlib.pyplot as plt
#from numba import jit
import pickle
from scipy.interpolate import interp1d
from torch.utils.data import DataLoader, random_split
import torch
from torchsummary import summary
import seaborn as sns
import sys
import torch.nn.functional as F
import pywt
from sklearn.preprocessing import MinMaxScaler
from torch import FloatTensor

# Req for package
sys.path.append("../")
from SkinLearning.Utils.NN import train, test, DEVICE


torch.backends.cudnn.benchmark = True

In [3]:
DEVICE = 'cpu'

In [5]:
# Folder name will correspond to index of sample
class SkinDataset(Dataset):
    def __init__(self, scaler, method="coefficients", combined=False, signalFolder="D:/SamplingResults2", sampleFile="../Data/newSamples.pkl", runs=range(65535), steps=128):
        # Load both disp1 and disp2 from each folder
        # Folders ordered according to index of sample
        # Use the corresponding sample as y -> append probe?
        self.input = []
        self.output = []
        
        with open(f"{sampleFile}", "rb") as f:
             samples = pickle.load(f)
        
        self.min = np.min(samples[runs])
        self.max = np.max(samples[runs])
        
        
        for run in tqdm(runs):
            inp = []
            fail = False
            
            files = os.listdir(f"{signalFolder}/{run}/")
            
            if files != ['Disp1.csv', 'Disp2.csv']:
                continue
            
            for file in files:
                a = pd.read_csv(f"{signalFolder}/{run}/{file}")
                a.rename(columns = {'0':'x', '0.1': 'y'}, inplace = True)
                
                # Skip if unconverged
                if a['x'].max() != 7.0:
                    fail = True
                    break

                # Interpolate curve for consistent x values
                xNew = np.linspace(0, 7, num=steps, endpoint=False)
                interped = interp1d(a['x'], a['y'], kind='cubic', fill_value="extrapolate")(xNew)
                    
                
                inp.append(interped.astype("float32"))
            
            if not fail:
                if len(inp) != 2:
                    raise Exception("sdf")

                self.input.append(inp)
                self.output.append(samples[int(run)])
        
        scaler.fit(self.output)
        self.output = scaler.fit_transform(self.output)
        self.output = tensor(self.output).type(FloatTensor)
        
        self.input = [waveletExtraction(sample, method, combined=combined) for sample in self.input]
        self.input = tensor(np.array(self.input)).type(FloatTensor)
        
        
    def __len__(self):
        return len(self.output)
    
    def __getitem__(self, idx):
        sample = {"input": self.input[idx], "output": self.output[idx]}
        return sample
    
    

In [6]:
"""
    Creates the data set from filtered samples
    Returns the dataset and the scaler
"""
def getDataset(**kwargs):
    # Get filtered data
    if not 'runs' in kwargs.keys():
        with open("../Data/filtered.pkl", "rb") as f:
            runs = pickle.load(f)

        kwargs['runs'] = runs

    scaler = MinMaxScaler()
    dataset = SkinDataset(scaler=scaler, **kwargs)

    return dataset, scaler

In [7]:
"""
    Creates a train/test split from the given data
    Returns train and test data loaders
"""
def getSplit(dataset, p1=0.8):
    train_n = int(p1 * len(dataset))
    test_n = len(dataset) - train_n
    train_set, test_set = random_split(dataset, [train_n, test_n])

    return DataLoader(train_set, batch_size=32, shuffle=True), \
        DataLoader(test_set, batch_size=32, shuffle=True)

In [8]:
datasets['non-combined']['raw'][0][0]['input']

NameError: name 'datasets' is not defined

In [72]:
from scipy.stats import skew, kurtosis, entropy
from math import gcd
from functools import reduce

def waveletExtraction(x, method, combined=False, wavelet='db4', level=6, combine_method="concatenate"):
    #s = MinMaxScaler()
    #x = s.fit_transform(x)
    if combined:
        coeffs = pywt.WaveletPacket(x, wavelet=wavelet, maxlevel=level, mode='symmetric')
        coeffs = extractCoeff(coeffs, level)
        
        
        features = []
        if method == "raw":
                features.append(coeffs)
        else:
            # Get sub-freqency band details
            for i in range(len(coeffs)):  
                    if method == "stats":
                        features.append(stats(coeffs[i]))
                    elif method == "entropy":
                        features.append(entropy(coeffs[i]))
                    elif method == "min-max":
                        features.append(minMax(coeffs[i]))
                    elif method=="energy":
                        features.append(getEnergy(coeffs[i]))
                    
        return features
    
        return combined_coefficients 
    else:
        wp1 = pywt.WaveletPacket(x[0], wavelet=wavelet, maxlevel=level)
        wp2 = pywt.WaveletPacket(x[1], wavelet=wavelet, maxlevel=level)
        
        coeffs1 = extractCoeff(wp1, 0)
        coeffs2 = extractCoeff(wp2, 0)

        features1, features2 = [], []
        
        if method == "raw":
                features1, features2 = coeffs1, coeffs2
        else:
            # Get sub-freqency band details
            for i in range(len(coeffs1)):
                if method == "stats":
                    features1.append(stats(coeffs1[i]))
                    features2.append(stats(coeffs2[i]))
                elif method == "entropy":
                    features1.append(entropy(coeffs1[i]))
                    features2.append(entropy(coeffs2[i]))
                elif method == "min-max":
                    features1.append(minMax(coeffs1[i]))
                    features2.append(minMax(coeffs2[i]))
                elif method=="energy":
                    features1.append(getEnergy(coeffs1[i]))
                    features2.append(getEnergy(coeffs2[i]))
        
        if combine_method == "concatenate":
            return [np.array(features1).flatten(), np.array(features2).flatten()]
        else:
            features1 = np.array(features1)
            features2 = np.array(features2)
            
            signal1_coeffs_transposed = features1.T
            signal2_coeffs_transposed = features2.T

            # Flatten the transposed arrays into 1D arrays
            signal1_coeffs_flattened = signal1_coeffs_transposed.flatten()
            signal2_coeffs_flattened = signal2_coeffs_transposed.flatten()

            # Interleave the flattened arrays element-wise
            combined_coeffs = np.empty(signal1_coeffs_flattened.shape[0] + signal2_coeffs_flattened.shape[0], dtype=signal1_coeffs_flattened.dtype)
            combined_coeffs[0::2] = signal1_coeffs_flattened
            combined_coeffs[1::2] = signal2_coeffs_flattened


            return combined_coeffs #np.concatenate(flattened) -> works better?

def minMax(coeffs):
    return [
        min(coeffs),
        max(coeffs)
    ]
    
def stats(c):
    return [
                np.mean(c),
                np.std(c),
                skew(c),
                kurtosis(c)
            ]

# Find the common denominator of the number of intervals for each level
def reorganise(features):
    # Calculate the number of time intervals for each level
    num_intervals = [len(feature) for feature in features]

    common_denominator = reduce(lambda x, y: x * y // gcd(x, y), num_intervals)

    # Initialize an empty list to store the reorganized features
    reorganized_features = []

    # Iterate through the time intervals and add the corresponding features to the reorganized list
    for t in range(common_denominator):
        interval_features = []

        for level_features, num_intervals_level in zip(features, num_intervals):
            
            # Find the corresponding feature index for the current time interval and level
            index = (t * num_intervals_level) // common_denominator
            interval_features.append(level_features[int(index)])

        reorganized_features.append(interval_features)
    
    return np.array(reorganized_features)

def extractCoeff(wp, level=0):
    coeffs = []
    coeffs =  wp.get_level(6, 'freq')
    features = np.array([c.data for c in coeffs])
    return features

def getEnergy(coefficients):
    energy = np.sum(np.square(coefficients))
    return energy

In [73]:
datasets = {}

for comb in ['non-combined']:
    datasets[comb] = {}
    
    for method in ['min-max', 'stats', 'raw', 'entropy', 'energy']:# , 'stats', 'raw', 'entropy']:
        datasets[comb][method] = getDataset(method=method)#, combined = True if comb == "combined" else False, runs=[1,2,3])

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:09<00:00, 226.88it/s]


In [75]:
datasets['non-combined']['energy'] = getDataset(method='energy')

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:10<00:00, 212.14it/s]


In [76]:
datasets['non-combined']['energy'][0][0]

{'input': tensor([[2.0634e-03, 1.9313e-04, 6.1960e-06, 1.3240e-05, 1.9456e-05, 6.7089e-06,
          1.2562e-05, 4.5536e-06, 6.8081e-06, 1.8880e-06, 2.1032e-06, 2.6671e-06,
          2.3956e-06, 1.5530e-06, 7.3691e-06, 1.3685e-05, 7.8473e-06, 2.9935e-06,
          3.7773e-07, 7.1285e-07, 8.2999e-07, 3.8769e-07, 1.7844e-07, 2.2033e-06,
          3.6354e-08, 2.0814e-07, 1.1370e-07, 1.0103e-06, 2.7930e-06, 1.3206e-06,
          5.9242e-07, 1.4897e-05, 5.1210e-09, 3.6606e-09, 5.7816e-10, 7.7265e-10,
          7.4486e-09, 3.2818e-09, 1.2351e-09, 4.3832e-09, 2.5644e-09, 9.9259e-10,
          1.8569e-09, 1.3191e-09, 2.6533e-09, 2.0654e-09, 1.2057e-09, 1.6858e-09,
          1.3479e-08, 4.2453e-09, 1.4717e-09, 9.9297e-09, 8.1621e-10, 3.5505e-10,
          1.3394e-09, 5.1120e-09, 4.2717e-08, 5.0222e-09, 8.4436e-09, 1.3365e-08,
          7.2233e-09, 8.0118e-10, 2.6555e-09, 2.6087e-07],
         [1.5781e-02, 2.4109e-03, 3.9224e-04, 3.6970e-04, 6.8843e-05, 3.5664e-06,
          2.4005e-05, 1.7359e-

In [77]:
loaders = {}
lengths = {}

for comb in datasets.keys():
    loaders[comb] = {}
    lengths[comb] = {}
    
    for method in datasets[comb].keys():
        loaders[comb][method] = {}
        loaders[comb][method]['train'], loaders[comb][method]['test'] = getSplit(datasets[comb][method][0])
        lengths[comb][method] = len(datasets[comb][method][0][0]['input'][0])

In [78]:
loaders, lengths

({'non-combined': {'min-max': {'train': <torch.utils.data.dataloader.DataLoader at 0x16d4b876860>,
    'test': <torch.utils.data.dataloader.DataLoader at 0x16e078f6020>},
   'stats': {'train': <torch.utils.data.dataloader.DataLoader at 0x16e078fd600>,
    'test': <torch.utils.data.dataloader.DataLoader at 0x16e078fdd80>},
   'raw': {'train': <torch.utils.data.dataloader.DataLoader at 0x16e078fca60>,
    'test': <torch.utils.data.dataloader.DataLoader at 0x16e078fc2b0>},
   'entropy': {'train': <torch.utils.data.dataloader.DataLoader at 0x16e078fda20>,
    'test': <torch.utils.data.dataloader.DataLoader at 0x16e078fcd60>},
   'energy': {'train': <torch.utils.data.dataloader.DataLoader at 0x16e078fe1d0>,
    'test': <torch.utils.data.dataloader.DataLoader at 0x16e078ff430>}}},
 {'non-combined': {'min-max': 128,
   'stats': 256,
   'raw': 512,
   'entropy': 64,
   'energy': 64}})

In [79]:
lengths2 = {}

for comb in datasets.keys():
    lengths2[comb] = {}
    
    for method in datasets[comb].keys():
        lengths2[comb][method] = len(datasets[comb][method][0][0]['input'][0])

In [80]:
lengths2

{'non-combined': {'min-max': 128,
  'stats': 256,
  'raw': 512,
  'entropy': 64,
  'energy': 64}}

In [105]:
class RNNMulti(nn.Module):
    def __init__(self, input_size=256, hidden_size=256, num_layers=1, method="concatenate"):
        super(RNNMulti, self).__init__()
        self.method = method
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        if method == 'concatenate':
            self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        elif method == 'multi_channel':
            self.rnn = nn.RNN(2, hidden_size, num_layers, batch_first=True)
        elif method == 'independent':
            self.rnn = nn.RNN(input_size//2, hidden_size, num_layers, batch_first=True)
        else:
            raise ValueError("Invalid method. Choose from 'concatenate', 'multi_channel', or 'independent'.")

        if method == 'independent':
            self.fc = nn.Linear(hidden_size*2, 6)
        else:
            self.fc = nn.Linear(hidden_size, 6)

    def forward(self, x):
        batch_size = x.size(0)
        signal_size = self.input_size//2
        signal1 = x[..., :signal_size]
        signal2 = x[..., signal_size:]
        
        x = x.reshape(batch_size, -1, self.input_size)
        signal1 = signal1.reshape(batch_size, -1, signal_size)
        signal2 = signal2.reshape(batch_size, -1, signal_size)
        

        if self.method == 'concatenate':
            _, hidden = self.rnn(x)
            hidden = hidden[-1]
        elif self.method == 'multi_channel':
            output, hidden = self.rnn(x.view(batch_size, -1, 2))
            hidden = hidden[-1]
        elif self.method == 'independent':
            output1, hidden1 = self.rnn(signal1)
            output2, hidden2 = self.rnn(signal2)
            hidden = torch.concat((hidden1[-1], hidden2[-1]), axis=0)

        output = self.fc(hidden)

        return output

In [100]:
class LSTMMulti(nn.Module):
    def __init__(self, input_size=256, hidden_size=256, num_layers=1, method="concatenate"):
        super(LSTMMulti, self).__init__()
        self.method = method
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        if method == 'concatenate':
            self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        elif method == 'multi_channel':
            self.lstm = nn.LSTM(2, hidden_size, num_layers, batch_first=True)
        elif method == 'independent':
            self.lstm = nn.LSTM(input_size//2, hidden_size, num_layers, batch_first=True)
        else:
            raise ValueError("Invalid method. Choose from 'concatenate', 'multi_channel', or 'independent'.")
        
        if method == 'independent':
            self.fc = nn.Linear(hidden_size*2, 6)
        else:
            self.fc = nn.Linear(hidden_size, 6)

    def forward(self, x):
        batch_size = x.size(0)
        signal_size = self.input_size//2
        signal1 = x[..., :signal_size]
        signal2 = x[..., signal_size:]
        
        x = x.reshape(batch_size, -1, self.input_size)
        signal1 = signal1.reshape(batch_size, -1, signal_size)
        signal2 = signal2.reshape(batch_size, -1, signal_size)

        h0 = torch.zeros((self.num_layers, batch_size, self.hidden_size)).to(x.device)  # Initialize hidden state to zeros
        c0 = torch.zeros((self.num_layers, batch_size, self.hidden_size)).to(x.device)  # Initialize cell state to zeros

        if self.method == 'concatenate':
            _, (hidden, cell) = self.lstm(x)
            hidden = hidden[-1]
        elif self.method == 'multi_channel':
            output, (hidden, cell) = self.lstm(x.view(batch_size, -1, 2))
            hidden = hidden[-1]
        elif self.method == 'independent':
            output1, (hidden1, cell1) = self.lstm(signal1)
            output2, (hidden2, cell2) = self.lstm(signal2)
            
            hidden = torch.concat((hidden1[-1], hidden2[-1]), axis=1)
            
        output = self.fc(hidden)

        return output

In [101]:
# Remove one FC LAyer
class RNNCurrent(nn.Module):
    def __init__(self):
        super(RNNCurrent, self).__init__()
        self.conv1 = nn.Conv1d(1, 128, kernel_size=5, padding=1, bias=False)
        self.pool1 = nn.MaxPool1d(kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm1d(128)
        
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.conv3 = nn.Conv1d(256, 512, kernel_size=3, padding=1, bias=False)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm1d(512)
        
        self.rnn = nn.RNN(1, 256, batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128 , 64),
            nn.ReLU(),
            nn.Linear(64, 6),   
        )

    def forward(self, x):
        a = x.shape
        batch_size = x.shape[0]
        
        x = x.reshape(batch_size, 1, -1)
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        
        
        out, h = self.rnn(x)
  
        x = out[:, -1, :]
        x = x.reshape(batch_size, -1)

        
        x = self.fc(x)
        
        x = x.view(batch_size, 6)
        return x

In [122]:
models = {}

# Get an LSTM and RNN of each type, paired with each type of dataset
for model_type in ['LSTM', 'RNN']:
    models[model_type] = {}
    
    for comb in datasets.keys():
        models[model_type][comb] = {}
        
        for ext_method in datasets[comb].keys():
            models[model_type][comb][ext_method] = {}
            
            for model_method in ['concatenate', 'multi_channel', 'independent']:
                models[model_type][comb][ext_method][model_method] = \
                LSTMMulti(method=model_method, input_size=lengths[comb][ext_method], hidden_size=2*lengths[comb][ext_method])

In [119]:
lengths

{'non-combined': {'min-max': 128,
  'stats': 256,
  'raw': 512,
  'entropy': 64,
  'energy': 64}}

In [130]:
losses = {}

for model_type in models.keys():
    losses[model_type] = {}
    if model_type == 'LSTM':
        continue
    for comb in models[model_type].keys():
        losses[model_type][comb] = {}
        
        for ext_method in models[model_type][comb].keys():
            losses[model_type][comb][ext_method] = {}
            
            for model_method in models[model_type][comb][ext_method].keys():
                print(f"Running {model_type} based on {model_method}, with dataset using {comb} {ext_method}")
                train_loss, val_loss = train(
                    loaders[comb][ext_method]['train'],
                    models[model_type][comb][ext_method][model_method],
                    val_loader=loaders[comb][ext_method]['test'],
                    LR=0.001,
                    epochs=1500, early_stopping=True)
                     

Running RNN based on concatenate, with dataset using non-combined min-max
Using: cuda:0


Epoch 68/1500: 100%|█████████████████████| 56/56 [00:00<00:00, 234.79batch/s, counter=2, lastLoss=0.115, valLoss=0.112]


Epoch 136/1500: 100%|██████████████████████| 56/56 [00:00<00:00, 243.47batch/s, counter=4, lastLoss=0.1, valLoss=0.102]


Epoch 204/1500: 100%|██████████████████| 56/56 [00:00<00:00, 242.94batch/s, counter=2, lastLoss=0.0936, valLoss=0.0953]


Epoch 272/1500: 100%|███████████████████| 56/56 [00:00<00:00, 235.79batch/s, counter=3, lastLoss=0.088, valLoss=0.0876]


...

Epoch 60/1500: 100%|████████████████████| 56/56 [00:00<00:00, 211.47batch/s, counter=49, lastLoss=0.186, valLoss=0.184]


Early stopping after 60 epochs
Average train loss: 0.007440499497134062
Average validation loss: 0.007371233327521216
Running RNN based on independent, with dataset using non-combined energy
Using: cuda:0


Epoch 68/1500: 100%|████████████████████| 56/56 [00:00<00:00, 204.96batch/s, counter=12, lastLoss=0.159, valLoss=0.155]


Epoch 105/1500: 100%|███████████████████| 56/56 [00:00<00:00, 209.79batch/s, counter=49, lastLoss=0.159, valLoss=0.163]


Early stopping after 105 epochs
Average train loss: 0.011660678441325824
Average validation loss: 0.01154915235340595


In [129]:
losses = {}
modelss = []

for comb in loaders.keys():
            for a in models[model_type][comb].keys():
                for ext_method in loaders[comb].keys():
                    print(f"Running {model_type} based on {model_method}, with dataset using {comb} {ext_method}")
                    modelss.append(SiameseRNN(input_size=lengths[comb][ext_method]))
                    train_loss, val_loss = train(
                        loaders[comb][ext_method]['train'],
                        modelss[-1],
                        val_loader=loaders[comb][ext_method]['test'],
                        LR=0.001,
                        epochs=300, early_stopping=True)

            print(test(loaders[comb][ext_method]['test'], m[-1], datasets[comb][ext_method][1]))


for i, method in enumerate(loaders['non-combined'].keys()):
    print(f"Running based on {method}")
    print(test(loaders['non-combined'][method]['test'], modelss[i], datasets['non-combined'][method][1]), "\n")

Running RNN based on independent, with dataset using non-combined min-max
Using: cuda:0


Epoch 11/300:  55%|████████████▋          | 31/56 [00:00<00:00, 96.72batch/s, counter=0, lastLoss=0.149, valLoss=0.147]


KeyboardInterrupt: 

In [126]:
for i, method in enumerate(loaders['non-combined'].keys()):
    print(f"Running based on {method}")
    print(test(loaders['non-combined'][method]['test'], modelss[i], datasets['non-combined'][method][1]), "\n")

Running based on min-max


100%|█████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 315.79 batch/s]

(90.46535587310791, array([94.535805, 76.38266 , 99.71    , 99.56899 , 83.740036, 88.854645],
      dtype=float32), 0.07161155641078949) 

Running based on stats


IndexError: list index out of range

In [ ]:
losses = {}

for model_type in models.keys():
    losses[model_type] = {}
    if model_type == 'LSTM':
        continue
    for comb in models[model_type].keys():
        losses[model_type][comb] = {}
        
        for ext_method in models[model_type][comb].keys():
            losses[model_type][comb][ext_method] = {}
            
            for model_method in models[model_type][comb][ext_method].keys():
                print(f"Running {model_type} based on {model_method}, with dataset using {comb} {ext_method}")
                m = SiameseRNN(input_size=lengths[comb][ext_method])
                train_loss, val_loss = train(
                    loaders[comb][ext_method]['train'],
                    m,
                    val_loader=loaders[comb][ext_method]['test'],
                    LR=0.001,
                    epochs=300, early_stopping=True)
                
                print(test(loaders[comb][ext_method]['test'], m, datasets[comb][ext_method][1]))
                
                

In [90]:
datasets['non-combined'].keys()

dict_keys(['min-max', 'stats', 'raw', 'entropy', 'energy'])

In [88]:
test(loaders[comb][ext_method]['test'], m, datasets[comb][ext_method][1])

100%|█████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 168.58 batch/s]


(91.24422645568848,
 array([94.51197 , 79.451096, 99.72503 , 99.62737 , 83.16258 , 90.987305],
       dtype=float32),
 0.06617546553413073)

In [67]:
losses = {}

for model_type in models.keys():
    losses[model_type] = {}
    if model_type == 'LSTM':
        continue
    for comb in models[model_type].keys():
        losses[model_type][comb] = {}
        
        for ext_method in models[model_type][comb].keys():
            losses[model_type][comb][ext_method] = {}
            
            for model_method in models[model_type][comb][ext_method].keys():
                print(f"Running {model_type} based on {model_method}, with dataset using {comb} {ext_method}")
                train_loss, val_loss = train(
                    loaders[comb][ext_method]['train'],
                    SiameseRNN(input_size=lengths[comb][ext_method]),
                    val_loader=loaders[comb][ext_method]['test'],
                    LR=0.001,
                    epochs=300, early_stopping=True)
                
                losses[model_type]['train'] = train_loss
                losses[model_type]['val'] = val_loss
                
                

Running RNN based on concatenate, with dataset using non-combined min-max
Using: cuda:0


Epoch 18/300: 100%|███████████████████████| 56/56 [00:01<00:00, 50.91batch/s, counter=0, lastLoss=0.133, valLoss=0.132]


KeyboardInterrupt: 

In [68]:
# In theory wont work
class SiameseRNN(nn.Module):
    def __init__(self, input_size=16, hidden_size=256):
        super(SiameseRNN, self).__init__()
        self.hidden_size = hidden_size
        # num_layers > 5 reduces performance
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=3, batch_first=True)
                
        self.fc = nn.Sequential(
            nn.Linear(hidden_size*2, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 6)
        
        )
    
    def forward(self, x):
        batch_size = x.shape[0]
        #x = x.reshape(batch_size, 1, -1)
        
        x1 = x[:, 0, :].reshape(batch_size, 1, -1)
        x2 = x[:, 1, :].reshape(batch_size, 1, -1)
        

        h0 = torch.zeros(5, batch_size, self.hidden_size).cpu()
        
        o1, h1 = self.rnn(x1)  # Add a batch dimension
        o2, h2 = self.rnn(x2)  # Add a batch dimension
        
        out = torch.cat([o1[:, -1, :], o2[:, -1, :]], dim=1)
        #out=h1[-1]
        out = out.reshape(batch_size, -1)
        out = self.fc(out)
        return out

In [29]:
def train(
    train_loader,
    net,
    LR=0.1,
    epochs=2000,
    val_loader=None,
    early_stopping=False,
    patience=50,
    optimizer=optim.Adam
):
    net.to(DEVICE)
    optimizer = optimizer(net.parameters(), lr=LR)
    criterion = nn.L1Loss()
    val_losses = []        
    losses = []
    last_loss = 0
    best_val_loss = 1e10
    counter = 0

    print(f"Using: {DEVICE}")
    
    for epoch in range(epochs):
        loss = 0
        net.train()
        with tqdm(train_loader, unit="batch") as it:
            if epoch > 0:
                it.set_postfix(lastLoss=last_loss, valLoss=0 if len(val_losses) \
                     == 0 else val_losses[-1], counter=counter)
            for idx, data in enumerate(it):
                it.set_description(f"Epoch {epoch+1}/{epochs}")
                inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)
                
                optimizer.zero_grad()
                predicted = net(inp)

                cost = criterion(out, predicted)
                loss += cost.item()
                cost.backward()
                optimizer.step()
        
        loss /= len(it)
        losses.append(loss)
        last_loss = loss
        
        if val_loader:
            val_loss = 0
            net.eval()
            for idx, data in enumerate(val_loader):
                inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)

                predicted = net(inp)
                cost = criterion(out, predicted)
                val_loss += cost.item()
            val_loss /= len(val_loader)  
            val_losses.append(val_loss)

            if early_stopping:
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    counter = 0
                else:
                    counter += 1
                if counter >= patience:
                    print(f"Early stopping after {epoch + 1} epochs")
                    break
         
    print(f"Average train loss: {np.sum(losses)/epochs}")
    print(f"Average validation loss: {np.sum(val_losses)/epochs}")
    
    return losses, val_losses